In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
import multiprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.utils import shuffle
from PIL import ImageFile
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.regularizers import l2
import os
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import Callback

In [2]:
# Enable loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Clear the previous session
K.clear_session()

style_train = '/Users/lucamullers/Downloads/wikiart/new_style_train_final.csv'
style_val = '/Users/lucamullers/Downloads/wikiart/new_style_val_final.csv'
image_directory = '/Users/lucamullers/Downloads/wikiart/'

# Load datasets
train_df = pd.read_csv(style_train)
val_df = pd.read_csv(style_val)

# Combine datasets
all_df = pd.concat([train_df, val_df], axis=0).reset_index(drop=True)

print(f"Before encoding: {len(all_df)} images")
all_labels = all_df['style_name'].str.get_dummies(sep='/')
print(f"After encoding: {len(all_labels)} labels (should match images)")

# Filter out styles with too few training samples
min_train_samples = 500
valid_classes = all_labels.sum(axis=0)[all_labels.sum(axis=0) >= min_train_samples].index
all_labels = all_labels[valid_classes]
all_df = all_df[all_labels.sum(axis=1) > 0]

# Ensure consistency between all_df and all_labels
all_labels = all_labels.loc[all_df.index].reset_index(drop=True)
all_df = all_df.reset_index(drop=True)

# Step 1: Split data - 10% Test Set
df_train_val, test_df, labels_train_val, test_labels = train_test_split(
    all_df, all_labels, test_size=0.1, random_state=42
)

# Step 2: Split remaining 90% into 70% Train and 20% Validation
train_df, val_df, train_labels, val_labels = train_test_split(
    df_train_val, labels_train_val, test_size=2/9, random_state=42
)

# Image size and batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 256

# Hybrid Sampling Function
def hybrid_sampling(df, labels):
    min_samples_per_class = 2000  # Only upsample classes below this
    max_samples_per_class = 3000  # Prevent large classes from growing

    image_counts = {}  # Track how often each image appears across all labels
    df_resampled = []
    labels_resampled = []

    # 1️⃣ **Ensure ONLY small classes (< 1200) get upsampled**
    for style in labels.columns:
        class_indices = labels[labels[style] == 1].index
        num_samples = len(class_indices)

        if num_samples < min_samples_per_class:
            resampled_indices = np.random.choice(class_indices, size=min_samples_per_class, replace=True)
        elif num_samples > max_samples_per_class:
            resampled_indices = np.random.choice(class_indices, size=max_samples_per_class, replace=False)
        else:
            resampled_indices = class_indices  # Keep mid-sized classes unchanged

        for idx in resampled_indices:
            img_path = df.loc[idx, 'image_path']

            # 2️⃣ **Prevent images from being overused**
            if img_path not in image_counts:
                image_counts[img_path] = 0
            image_counts[img_path] += 1

            if image_counts[img_path] <= 10:  # Allow max 2 occurrences per image
                df_resampled.append(df.loc[idx])
                labels_resampled.append(labels.loc[idx])

    # 3️⃣ **Convert lists back to DataFrame after resampling**
    resampled_df = pd.DataFrame(df_resampled).reset_index(drop=True)
    resampled_labels = pd.DataFrame(labels_resampled, columns=labels.columns).reset_index(drop=True)

    return shuffle(resampled_df, resampled_labels, random_state=42)

# Data Generators
def generator_with_labels(datagen, dataframe, labels, batch_size):
    num_samples = len(dataframe)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for start in range(0, num_samples, batch_size):
            end = min(start + batch_size, num_samples)
            batch_indices = indices[start:end]
            batch_images = []
            for idx in batch_indices:
                img_path = dataframe.iloc[idx]['image_path']
                img = tf.keras.preprocessing.image.load_img(f"{image_directory}/{img_path}", target_size=IMAGE_SIZE)
                img = tf.keras.preprocessing.image.img_to_array(img)
                img = datagen.random_transform(img)
                img = datagen.standardize(img)
                batch_images.append(img)
            batch_images = np.array(batch_images)
            batch_labels = labels.iloc[batch_indices].values
            yield batch_images, batch_labels

# Define Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1./255)

# Apply sampling to check actual image and label counts 
sampled_df, sampled_labels = hybrid_sampling(train_df, train_labels)

# Print class distribution after sampling
print("\nClass counts after resampling:")
print(sampled_labels.sum().sort_values(ascending=False))

# Print total number of training samples
print(f"\nTotal resampled training samples: {len(sampled_df)}")

Before encoding: 81446 images
After encoding: 81446 labels (should match images)

Class counts after resampling:
Realism_and_19th_Century_Movements      19000
Renaissance_and_Mannerism               11000
Impressionism_and_Post_Impressionism    10989
Expressionism_and_Derivatives           10002
Modern_Art                               9000
Baroque_and_Rococo                       8000
Baroque                                  5047
Impressionism                            4909
Expressionism                            4907
Realism                                  4037
Post_Impressionism                       4010
Romanticism                              3721
Art_Nouveau                              3458
Symbolism                                3456
Northern_Renaissance                     3157
Rococo                                   2953
Color_Field_Painting                     2908
Pop_Art                                  2866
Abstract_Expressionism                   2815
Minimalism   

In [3]:
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Multiply, Reshape
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K


# === Resample training set ===
train_df_resampled, train_labels_resampled = hybrid_sampling(train_df, train_labels)

# === Create generators ===
train_gen = generator_with_labels(train_datagen, train_df_resampled, train_labels_resampled, BATCH_SIZE)
val_gen = generator_with_labels(val_datagen, val_df, val_labels, BATCH_SIZE)

# === Compute steps ===
steps_per_epoch = int(np.ceil(len(train_df_resampled) / BATCH_SIZE))
validation_steps = int(np.ceil(len(val_df) / BATCH_SIZE))

# === Learning rate schedule ===
lr_schedule = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=steps_per_epoch * 10,
    decay_rate=0.96,
    staircase=True
)

L2_FACTOR = 0.001
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[-100:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)  # shape: (None, 2048)
x = Dense(1024, activation='relu', kernel_regularizer=l2(L2_FACTOR))(x)  # (None, 1024)

def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')  # <-- Add this line
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    return loss


# === Gating mechanism: one gate per style ===
num_styles = train_labels.shape[1]
gates = Dense(num_styles * 1024, activation='sigmoid', name='gating_layer')(x)
gates = Reshape((num_styles, 1024))(gates)        # shape: (None, num_styles, 2048)
x_repeated = tf.keras.layers.RepeatVector(num_styles)(x)  # shape: (None, num_styles, 2048)
x_filtered = Multiply()([x_repeated, gates])      # shape: (None, num_styles, 2048)

# === Final classifier ===
logits = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=l2(L2_FACTOR))
outputs = tf.keras.layers.TimeDistributed(logits)(x_filtered)  # shape: (None, num_styles, 1)
style_output = tf.keras.layers.Reshape((num_styles,))(outputs)

model = Model(inputs=base_model.input, outputs=style_output)

# === Compile model ===
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=[
        'accuracy',
        AUC(name='auc', multi_label=True),
        Precision(name='precision'),
        Recall(name='recall')
    ]
)

class SaveEachEpoch(Callback):
    def __init__(self, save_path):
        super().__init__()
        self.save_path = save_path
        os.makedirs(self.save_path, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        path = os.path.join(self.save_path, f'model_epoch_{epoch:02d}.h5')
        self.model.save(path)
        print(f'\nSaved model at {path}')

save_callback = SaveEachEpoch(save_path='checkpoints_final_focal_testing_2/')

# === Train model ===
history = model.fit(
    train_gen,
    epochs=30,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_gen,
    validation_steps=validation_steps,
    verbose=1,
    callbacks=[save_callback]
)


Epoch 1/30
 43/274 ━━━━━━━━━━━━━━━━━━━━ 1:53:16 29s/step - accuracy: 0.1062 - auc: 0.5048 - loss: 1.3788 - precision: 0.0705 - recall: 0.0435

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (107327830 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


 97/274 ━━━━━━━━━━━━━━━━━━━━ 1:25:33 29s/step - accuracy: 0.1223 - auc: 0.5329 - loss: 1.3165 - precision: 0.0899 - recall: 0.0246

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/PIL/Image.py:3182: DecompressionBombWarning: Image size (99962094 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - accuracy: 0.1565 - auc: 0.6592 - loss: 1.1313 - precision: 0.3809 - recall: 0.0439 

...


Saved model at checkpoints_final_focal_testing_2/model_epoch_25.h5
274/274 ━━━━━━━━━━━━━━━━━━━━ 7969s 29s/step - accuracy: 0.4248 - auc: 0.9969 - loss: 0.0039 - precision: 0.9745 - recall: 0.8617 - val_accuracy: 0.2711 - val_auc: 0.9226 - val_loss: 0.0269 - val_precision: 0.6705 - val_recall: 0.5232
Epoch 27/30
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.4083 - auc: 0.9972 - loss: 0.0037 - precision: 0.9766 - recall: 0.8688 


Saved model at checkpoints_final_focal_testing_2/model_epoch_26.h5
274/274 ━━━━━━━━━━━━━━━━━━━━ 7931s 29s/step - accuracy: 0.4083 - auc: 0.9972 - loss: 0.0037 - precision: 0.9766 - recall: 0.8688 - val_accuracy: 0.3008 - val_auc: 0.9236 - val_loss: 0.0259 - val_precision: 0.6893 - val_recall: 0.5489
Epoch 28/30
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.4077 - auc: 0.9974 - loss: 0.0037 - precision: 0.9767 - recall: 0.8735 


Saved model at checkpoints_final_focal_testing_2/model_epoch_27.h5
274/274 ━━━━━━━━━━━━━━━━━━━━ 7922s 29s/step - accuracy: 0.4077 - auc: 0.9974 - loss: 0.0037 - precision: 0.9767 - recall: 0.8735 - val_accuracy: 0.2897 - val_auc: 0.9271 - val_loss: 0.0243 - val_precision: 0.7175 - val_recall: 0.5669
Epoch 29/30
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.4044 - auc: 0.9974 - loss: 0.0036 - precision: 0.9764 - recall: 0.8764 


Saved model at checkpoints_final_focal_testing_2/model_epoch_28.h5
274/274 ━━━━━━━━━━━━━━━━━━━━ 7957s 29s/step - accuracy: 0.4044 - auc: 0.9974 - loss: 0.0036 - precision: 0.9764 - recall: 0.8764 - val_accuracy: 0.2449 - val_auc: 0.9227 - val_loss: 0.0237 - val_precision: 0.6995 - val_recall: 0.5352
Epoch 30/30
274/274 ━━━━━━━━━━━━━━━━━━━━ 0s 29s/step - accuracy: 0.4079 - auc: 0.9977 - loss: 0.0035 - precision: 0.9773 - recall: 0.8799 


Saved model at checkpoints_final_focal_testing_2/model_epoch_29.h5
274/274 ━━━━━━━━━━━━━━━━━━━━ 8218s 30s/step - accuracy: 0.4080 - auc: 0.9977 - loss: 0.0035 - precision: 0.9773 - recall: 0.8799 - val_accuracy: 0.2685 - val_auc: 0.9207 - val_loss: 0.0260 - val_precision: 0.6869 - val_recall: 0.5500


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
import tensorflow.keras.backend as K
import scipy

# === Loss function used during training ===
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    loss.__name__ = 'focal_loss'
    return loss

# === Validation generator ===
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory=image_directory,
    x_col='image_path',
    y_col=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)
val_steps = int(np.ceil(len(val_df) / BATCH_SIZE))

# === Evaluate selected checkpoints ===
EVALUATE_EPOCHS = list(range(30))  # Modify as needed
checkpoint_dir = "checkpoints_final_focal_testing_2/"
threshold = 0.3  # Modify as needed

for epoch in EVALUATE_EPOCHS:
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch:02d}.h5")
    if not os.path.exists(checkpoint_path):
        print(f"Skipped epoch {epoch} (not found)")
        continue

    print(f"\n=== Evaluating Epoch {epoch+1} ===")
    
    model = load_model(checkpoint_path, custom_objects={'loss': focal_loss(gamma=2.0, alpha=0.25)})

    val_predictions = model.predict(val_generator, steps=val_steps)
    val_predictions = val_predictions[:len(val_df)]
    val_predicted_labels = (val_predictions > threshold).astype(int)

    val_auc = roc_auc_score(val_labels, val_predictions, average='macro')
    val_f1 = f1_score(val_labels, val_predicted_labels, average='macro', zero_division=0)
    val_precision = precision_score(val_labels, val_predicted_labels, average='macro', zero_division=0)
    val_recall = recall_score(val_labels, val_predicted_labels, average='macro', zero_division=0)

    print(f"Validation AUC: {val_auc:.4f}")
    print(f"Validation F1-Score: {val_f1:.4f}")
    print(f"Validation Precision: {val_precision:.4f}")
    print(f"Validation Recall: {val_recall:.4f}")


Found 16290 validated image filenames.

=== Evaluating Epoch 1 ===


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


64/64 ━━━━━━━━━━━━━━━━━━━━ 282s 4s/step
Validation AUC: 0.4654
Validation F1-Score: 0.0144
Validation Precision: 0.0563
Validation Recall: 0.0980

=== Evaluating Epoch 2 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 280s 4s/step
Validation AUC: 0.6944
Validation F1-Score: 0.1133
Validation Precision: 0.2404
Validation Recall: 0.1939

=== Evaluating Epoch 3 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 281s 4s/step
Validation AUC: 0.8720
Validation F1-Score: 0.3918
Validation Precision: 0.4351
Validation Recall: 0.4605

=== Evaluating Epoch 4 ===


...

64/64 ━━━━━━━━━━━━━━━━━━━━ 275s 4s/step
Validation AUC: 0.9224
Validation F1-Score: 0.5735
Validation Precision: 0.5548
Validation Recall: 0.6361

=== Evaluating Epoch 27 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 278s 4s/step
Validation AUC: 0.9245
Validation F1-Score: 0.5892
Validation Precision: 0.5805
Validation Recall: 0.6213

=== Evaluating Epoch 28 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 275s 4s/step
Validation AUC: 0.9273
Validation F1-Score: 0.5993
Validation Precision: 0.6068
Validation Recall: 0.6099

=== Evaluating Epoch 29 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 276s 4s/step
Validation AUC: 0.9228
Validation F1-Score: 0.5939
Validation Precision: 0.5855
Validation Recall: 0.6252

=== Evaluating Epoch 30 ===


64/64 ━━━━━━━━━━━━━━━━━━━━ 276s 4s/step
Validation AUC: 0.9210
Validation F1-Score: 0.5816
Validation Precision: 0.5950
Validation Recall: 0.6003


In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (
    f1_score, roc_auc_score, precision_score, recall_score,
    classification_report, multilabel_confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K

# === Define focal loss (must match training) ===
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, dtype='float32')
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        return K.mean(weight * cross_entropy)
    loss.__name__ = 'focal_loss'
    return loss

# === Setup ===
threshold = 0.32
checkpoint_dir = "checkpoints_final_focal_testing_2/"
EVALUATE_EPOCHS = list(range(30))

# === Test generator ===
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=image_directory,
    x_col='image_path',
    y_col=None,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False
)
test_steps = int(np.ceil(len(test_df) / BATCH_SIZE))

# === Evaluate each model ===
for epoch in EVALUATE_EPOCHS:
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch:02d}.h5")
    if not os.path.exists(checkpoint_path):
        print(f"Skipped epoch {epoch} (not found)")
        continue

    print(f"\n=== Evaluating Epoch {epoch + 1} on Test Set ===")

    model = load_model(checkpoint_path, custom_objects={'loss': focal_loss(gamma=2.0, alpha=0.25)})

    test_predictions = model.predict(test_generator, steps=test_steps)
    test_predictions = test_predictions[:len(test_df)]
    test_predicted_labels = (test_predictions > threshold).astype(int)

    # === Metrics ===
    test_auc = roc_auc_score(test_labels, test_predictions, average='macro')
    test_f1 = f1_score(test_labels, test_predicted_labels, average='macro', zero_division=0)
    test_precision = precision_score(test_labels, test_predicted_labels, average='macro', zero_division=0)
    test_recall = recall_score(test_labels, test_predicted_labels, average='macro', zero_division=0)

    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test F1-Score: {test_f1:.4f}")
    print(f"Test Precision: {test_precision:.4f}")
    print(f"Test Recall: {test_recall:.4f}")

    print("\nClassification Report:")
    print(classification_report(
        test_labels, 
        test_predicted_labels, 
        target_names=valid_classes, 
        zero_division=0
    ))

Found 8145 validated image filenames.

=== Evaluating Epoch 1 on Test Set ===


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step
Test AUC: 0.4623
Test F1-Score: 0.0120
Test Precision: 0.0317
Test Recall: 0.0813

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.00      0.00      0.00       246
                         Art_Nouveau       0.00      0.00      0.00       425
                             Baroque       0.00      0.00      0.00       400
                  Baroque_and_Rococo       0.00      0.00      0.00       615
                Color_Field_Painting       0.04      0.12      0.06       144
                              Cubism       0.00      0.00      0.00       230
                   Early_Renaissance       0.00      0.00      0.00       139
                       Expressionism       0.00      0.00      0.00       685
       Expressionism_and_Derivatives       0.10      0.16      0.13      1047
                             Fauvism       0.00      0.00      0.00      

32/32 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step
Test AUC: 0.6983
Test F1-Score: 0.0988
Test Precision: 0.2109
Test Recall: 0.1642

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.07      0.41      0.12       246
                         Art_Nouveau       0.40      0.15      0.22       425
                             Baroque       0.21      0.02      0.04       400
                  Baroque_and_Rococo       0.32      0.03      0.05       615
                Color_Field_Painting       0.09      0.36      0.14       144
                              Cubism       0.95      0.08      0.14       230
                   Early_Renaissance       0.75      0.02      0.04       139
                       Expressionism       0.31      0.09      0.14       685
       Expressionism_and_Derivatives       0.23      0.50      0.31      1047
                             Fauvism       0.00      0.00      0.00      

32/32 ━━━━━━━━━━━━━━━━━━━━ 139s 4s/step
Test AUC: 0.8711
Test F1-Score: 0.3731
Test Precision: 0.4690
Test Recall: 0.4098

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.36      0.47      0.41       246
                         Art_Nouveau       0.27      0.48      0.35       425
                             Baroque       0.23      0.68      0.34       400
                  Baroque_and_Rococo       0.31      0.72      0.44       615
                Color_Field_Painting       0.62      0.65      0.64       144
                              Cubism       0.76      0.33      0.46       230
                   Early_Renaissance       0.35      0.60      0.45       139
                       Expressionism       0.40      0.34      0.37       685
       Expressionism_and_Derivatives       0.42      0.62      0.50      1047
                             Fauvism       0.31      0.25      0.27      

...

32/32 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step
Test AUC: 0.9217
Test F1-Score: 0.5788
Test Precision: 0.5719
Test Recall: 0.6266

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.49      0.61      0.54       246
                         Art_Nouveau       0.28      0.78      0.42       425
                             Baroque       0.50      0.77      0.60       400
                  Baroque_and_Rococo       0.60      0.84      0.70       615
                Color_Field_Painting       0.69      0.74      0.71       144
                              Cubism       0.57      0.58      0.57       230
                   Early_Renaissance       0.56      0.76      0.64       139
                       Expressionism       0.50      0.48      0.49       685
       Expressionism_and_Derivatives       0.61      0.57      0.59      1047
                             Fauvism       0.39      0.14      0.20      

32/32 ━━━━━━━━━━━━━━━━━━━━ 146s 4s/step
Test AUC: 0.9243
Test F1-Score: 0.5851
Test Precision: 0.5875
Test Recall: 0.6059

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.53      0.58      0.56       246
                         Art_Nouveau       0.39      0.72      0.50       425
                             Baroque       0.55      0.66      0.60       400
                  Baroque_and_Rococo       0.69      0.72      0.71       615
                Color_Field_Painting       0.62      0.76      0.68       144
                              Cubism       0.51      0.67      0.58       230
                   Early_Renaissance       0.54      0.78      0.64       139
                       Expressionism       0.39      0.57      0.46       685
       Expressionism_and_Derivatives       0.53      0.64      0.58      1047
                             Fauvism       0.36      0.18      0.24      

32/32 ━━━━━━━━━━━━━━━━━━━━ 148s 5s/step
Test AUC: 0.9256
Test F1-Score: 0.5998
Test Precision: 0.6262
Test Recall: 0.5961

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.53      0.53      0.53       246
                         Art_Nouveau       0.46      0.64      0.53       425
                             Baroque       0.54      0.74      0.63       400
                  Baroque_and_Rococo       0.71      0.76      0.73       615
                Color_Field_Painting       0.76      0.65      0.70       144
                              Cubism       0.77      0.47      0.59       230
                   Early_Renaissance       0.91      0.52      0.66       139
                       Expressionism       0.46      0.51      0.48       685
       Expressionism_and_Derivatives       0.57      0.58      0.58      1047
                             Fauvism       0.40      0.17      0.24      

32/32 ━━━━━━━━━━━━━━━━━━━━ 150s 5s/step
Test AUC: 0.9255
Test F1-Score: 0.5916
Test Precision: 0.5894
Test Recall: 0.6136

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.57      0.54      0.55       246
                         Art_Nouveau       0.58      0.53      0.56       425
                             Baroque       0.63      0.59      0.61       400
                  Baroque_and_Rococo       0.74      0.68      0.71       615
                Color_Field_Painting       0.70      0.72      0.71       144
                              Cubism       0.52      0.63      0.57       230
                   Early_Renaissance       0.67      0.61      0.64       139
                       Expressionism       0.38      0.62      0.47       685
       Expressionism_and_Derivatives       0.48      0.70      0.57      1047
                             Fauvism       0.27      0.31      0.29      

32/32 ━━━━━━━━━━━━━━━━━━━━ 146s 5s/step
Test AUC: 0.9241
Test F1-Score: 0.5861
Test Precision: 0.6139
Test Recall: 0.5917

Classification Report:
                                      precision    recall  f1-score   support

              Abstract_Expressionism       0.50      0.53      0.52       246
                         Art_Nouveau       0.35      0.76      0.48       425
                             Baroque       0.62      0.57      0.60       400
                  Baroque_and_Rococo       0.79      0.63      0.70       615
                Color_Field_Painting       0.64      0.74      0.69       144
                              Cubism       0.82      0.34      0.48       230
                   Early_Renaissance       0.77      0.61      0.68       139
                       Expressionism       0.39      0.57      0.46       685
       Expressionism_and_Derivatives       0.51      0.67      0.58      1047
                             Fauvism       0.40      0.27      0.32      